# MLflow 실험 트래킹 — P2T2 파이프라인 결과 로깅

> **Purpose:** Gold 임상 요약, SOAP 노트, BioMedCLIP 매칭, Judge 평가 결과를
> MLflow Experiment에 기록하여 모델 성능을 추적합니다.
>
> **카탈로그:** `P2T2`  
> **소스 테이블:** `gold.patient_clinical_summary`, `ai_results.*`  
> **MLflow 실험:** `/Shared/P2T2_*`

## 0. 카탈로그 및 MLflow 설정

In [ ]:

import mlflow
from mlflow.tracking import MlflowClient
from pyspark.sql import functions as F
from datetime import datetime
import json
import tempfile
import os

spark.sql("USE CATALOG P2T2")

client = MlflowClient()

# ── 실험 정의 (플랫 경로 — 중첩 폴더 불필요) ──
EXPERIMENTS = {
    "pipeline_metrics": "/Shared/P2T2_Pipeline_Metrics",
    "clinical_inference": "/Shared/P2T2_Clinical_Inference",
    "judge_evaluation": "/Shared/P2T2_Judge_Evaluation",
}

for key, name in EXPERIMENTS.items():
    exp = mlflow.set_experiment(name)
    print(f"✅ 실험 준비: {name} (ID: {exp.experiment_id})")

print(f"\n🔧 MLflow Tracking URI: {mlflow.get_tracking_uri()}")

## 1. Run 1 — Gold Pipeline 메트릭

환자별 임상 요약 통계를 MLflow에 기록합니다.

In [ ]:

# Gold 테이블 읽기
df_gold = spark.table("P2T2.gold.patient_clinical_summary")
gold_cols = df_gold.columns
gold_rows = df_gold.collect()

mlflow.set_experiment(EXPERIMENTS["pipeline_metrics"])

for row in gold_rows:
    pid = row["patient_id"]
    
    with mlflow.start_run(run_name=f"gold_summary_{pid}"):
        mlflow.log_params({
            "patient_id": pid,
            "pipeline_stage": "gold",
            "source_table": "P2T2.gold.patient_clinical_summary",
        })
        
        metrics = {}
        metric_cols = ["avg_heart_rate", "avg_systolic_bp", "avg_diastolic_bp",
                       "avg_spo2", "avg_temperature", "avg_respiratory_rate",
                       "max_risk_score", "avg_risk_score"]
        for col in metric_cols:
            if col in gold_cols:
                val = row[col]
                if val is not None:
                    metrics[col] = float(val)
        
        for cnt_col in ["vital_count", "history_count"]:
            if cnt_col in gold_cols and row[cnt_col] is not None:
                metrics[cnt_col] = int(row[cnt_col])
        
        if metrics:
            mlflow.log_metrics(metrics)
        
        mlflow.set_tags({"project": "P2T2", "phase": "gold_aggregation", "patient_id": pid})
        
        run_id = mlflow.active_run().info.run_id
        print(f"✅ Gold [{pid}] logged — run_id: {run_id}")
        for k, v in metrics.items():
            print(f"   {k} = {v}")

## 2. Run 2 — BioMedCLIP 영상 분석

BioMedCLIP 영상-텍스트 매칭 결과를 기록합니다.

In [ ]:

df_clip = spark.table("P2T2.ai_results.biomedclip_results")
clip_cols = df_clip.columns
clip_rows = df_clip.collect()

mlflow.set_experiment(EXPERIMENTS["clinical_inference"])

for row in clip_rows:
    pid = row["patient_id"]
    
    with mlflow.start_run(run_name=f"biomedclip_{pid}"):
        mlflow.log_params({
            "patient_id": pid,
            "model_name": "microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224",
            "model_type": "multimodal_clip",
            "source_table": "P2T2.ai_results.biomedclip_results",
        })
        
        if "top_similarity" in clip_cols:
            sim_val = row["top_similarity"]
            if sim_val is not None:
                try:
                    mlflow.log_metric("top_similarity", float(sim_val))
                except (ValueError, TypeError):
                    mlflow.log_param("top_similarity_str", str(sim_val)[:250])
        
        tags = {"project": "P2T2", "phase": "ai_inference"}
        if "top_diagnosis" in clip_cols:
            tags["top_diagnosis"] = str(row["top_diagnosis"])[:250]
        if "urgency_level" in clip_cols:
            tags["urgency_level"] = str(row["urgency_level"])
        mlflow.set_tags(tags)
        
        run_id = mlflow.active_run().info.run_id
        print(f"✅ BioMedCLIP [{pid}] logged — run_id: {run_id}")

## 3. Run 3 — OpenAI SOAP 노트 생성

GPT-5.1 기반 SOAP 노트 생성 결과를 기록합니다.

In [ ]:

df_soap = spark.table("P2T2.ai_results.openai_soap_notes")
soap_cols = df_soap.columns
soap_rows = df_soap.collect()

mlflow.set_experiment(EXPERIMENTS["clinical_inference"])

for row in soap_rows:
    pid = row["patient_id"]
    
    with mlflow.start_run(run_name=f"soap_note_{pid}"):
        params = {
            "patient_id": pid,
            "model_type": "llm_soap_generation",
            "source_table": "P2T2.ai_results.openai_soap_notes",
        }
        if "model_version" in soap_cols:
            params["model_version"] = str(row["model_version"])
        mlflow.log_params(params)
        
        if "tokens_used" in soap_cols:
            tokens = row["tokens_used"]
            if tokens is not None:
                try:
                    mlflow.log_metric("tokens_used", float(tokens))
                except (ValueError, TypeError):
                    mlflow.log_param("tokens_used_str", str(tokens))
        
        soap_text = ""
        if "soap_note" in soap_cols:
            soap_text = str(row["soap_note"] or "")
        mlflow.log_metric("soap_length_chars", len(soap_text))
        mlflow.log_metric("soap_length_words", len(soap_text.split()))
        
        if soap_text:
            with tempfile.NamedTemporaryFile(mode="w", suffix=".txt", delete=False, encoding="utf-8") as f:
                f.write(f"Patient: {pid}\n")
                f.write("=" * 60 + "\n\n")
                f.write(soap_text)
                tmp_path = f.name
            mlflow.log_artifact(tmp_path, artifact_path="soap_notes")
            os.unlink(tmp_path)
        
        mlflow.set_tags({"project": "P2T2", "phase": "soap_generation", "patient_id": pid})
        
        run_id = mlflow.active_run().info.run_id
        print(f"✅ SOAP [{pid}] logged — run_id: {run_id}, {len(soap_text)} chars")

## 4. Run 4 — LLM-as-a-Judge 평가

Judge 평가 점수를 기록합니다.

In [ ]:

df_judge = spark.table("P2T2.ai_results.judge_evaluation")
judge_cols = df_judge.columns
judge_rows = df_judge.collect()

mlflow.set_experiment(EXPERIMENTS["judge_evaluation"])

for row in judge_rows:
    pid = row["patient_id"]
    
    with mlflow.start_run(run_name=f"judge_eval_{pid}"):
        params = {
            "patient_id": pid,
            "model_type": "llm_as_a_judge",
            "source_table": "P2T2.ai_results.judge_evaluation",
        }
        if "judge_model" in judge_cols:
            params["judge_model"] = str(row["judge_model"])
        mlflow.log_params(params)
        
        if "overall_score" in judge_cols:
            overall = row["overall_score"]
            if overall is not None:
                try:
                    mlflow.log_metric("overall_score", float(overall))
                except (ValueError, TypeError):
                    mlflow.log_param("overall_score_str", str(overall))
        
        if "confidence" in judge_cols:
            conf = row["confidence"]
            if conf is not None:
                try:
                    mlflow.log_metric("confidence", float(conf))
                except (ValueError, TypeError):
                    mlflow.log_param("confidence_str", str(conf))
        
        if "evaluation_json" in judge_cols:
            eval_json = row["evaluation_json"]
            if eval_json:
                try:
                    eval_data = json.loads(eval_json) if isinstance(eval_json, str) else eval_json
                    for k, v in eval_data.items():
                        if isinstance(v, (int, float)):
                            mlflow.log_metric(f"judge_{k}", float(v))
                    
                    with tempfile.NamedTemporaryFile(mode="w", suffix=".json", delete=False, encoding="utf-8") as f:
                        json.dump(eval_data, f, ensure_ascii=False, indent=2)
                        tmp_path = f.name
                    mlflow.log_artifact(tmp_path, artifact_path="judge_details")
                    os.unlink(tmp_path)
                except (json.JSONDecodeError, TypeError):
                    mlflow.log_param("evaluation_raw", str(eval_json)[:250])
        
        tags = {"project": "P2T2", "phase": "judge_evaluation", "patient_id": pid}
        if "pass_fail" in judge_cols:
            tags["pass_fail"] = str(row["pass_fail"])
        mlflow.set_tags(tags)
        
        run_id = mlflow.active_run().info.run_id
        overall_val = row["overall_score"] if "overall_score" in judge_cols else "N/A"
        pass_fail = row["pass_fail"] if "pass_fail" in judge_cols else "N/A"
        print(f"✅ Judge [{pid}] logged — run_id: {run_id}")
        print(f"   Score: {overall_val}/5, Pass/Fail: {pass_fail}")

## 5. 실험 요약 리포트

In [ ]:

print("=" * 70)
print("📊 P2T2 MLflow 실험 등록 요약")
print("=" * 70)

total_runs = 0
for key, name in EXPERIMENTS.items():
    exp = client.get_experiment_by_name(name)
    if exp:
        runs = client.search_runs(experiment_ids=[exp.experiment_id])
        total_runs += len(runs)
        print(f"\n  📁 {name}")
        print(f"     Runs: {len(runs)}")
        for r in runs:
            run_name = r.data.tags.get("mlflow.runName", "unnamed")
            metrics_str = ", ".join(
                f"{k}={v:.3f}" for k, v in sorted(r.data.metrics.items())[:5]
            )
            print(f"     • {run_name}: {metrics_str}")
    else:
        print(f"\n  ⚪ {name}: 실험 없음")

print(f"\n{'=' * 70}")
print(f"  Total: {total_runs} runs across {len(EXPERIMENTS)} experiments")
print(f"  MLflow UI: {mlflow.get_tracking_uri()}")
print("=" * 70)